**Описание данных**

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем библ которые могут понадобиться.

In [ ]:
import pandas as pd
import numpy as np
import nltk
import torch

from nltk.corpus import stopwords
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

import lightgbm as lgb
import xgboost as xgb

from sklearn.metrics import  f1_score
from sklearn.metrics import classification_report, roc_auc_score


import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
df = pd.read_csv('/content/toxic_comments.csv')

In [ ]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим на наличие пропусков и дубликатов.

In [ ]:
print(df.isna().mean(), 'Дубликаты: {}'.format(df.duplicated().sum()), sep='\n')

text     0.0
toxic    0.0
dtype: float64
Дубликаты: 0


Посмотрим на общую инфу и распределение.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
df.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Данные впорядке. Приступим к подготовки текста.

Подгрузим необходимые словари и обработаем наш текст и удалим от туда не нужные символы. Сохраним все в столбец в датасет.

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
wnl = WordNetLemmatizer()

def clear_text(txt):
  text = ' '.join(re.sub(r'[^a-zA-Z ]', ' ', txt).split())
  tokens = word_tokenize(text)
  text = [wnl.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else wnl.lemmatize(i) for i,j in pos_tag(tokens)]
  text = ' '.join(text)
  return text

In [ ]:
df['lemm_text'] = df['text'].apply(clear_text)

Сравним наш lemm_text и text

In [ ]:
print('Старый тест: {}'.format(df['text'][0]))
print()
print('Обработанный тест:', df['lemm_text'][0], sep='\n')

Старый тест: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Обработанный тест:
Explanation Why the edits make under my username Hardcore Metallica Fan be revert They weren t vandalism just closure on some GAs after I vote at New York Dolls FAC And please don t remove the template from the talk page since I m retire now


## Вывод
Обработали тест, теперь он имеет просто слова без лишнего мусора в виде символов. Можно приступать к обучению моделей.

## Обучение

Прежде разобьем наши данные на обучающую и тестовую выборку с сохранением баланса классов в них.

In [ ]:
train, test = train_test_split(df, random_state=12345, test_size=0.3, stratify=df['toxic'])

Теперь мы создадим корпуса выборок с признаками и отдельно целевой признак.

In [ ]:
corpus_features_train = list(train['lemm_text'])
target_train = train['toxic']

corpus_features_test = list(test['lemm_text'])
target_test = test['toxic']

Подгрузим стоп слова и передадим их нашей TF-IDF, чтобы избавиться от не нужных для контекста слов.

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Чтобы посчитать TF-IDF для корпуса текстов вызовем и обучим на train.

In [ ]:
tf_idf_train = count_tf_idf.fit_transform(corpus_features_train)
tf_idf_test = count_tf_idf.transform(corpus_features_test)

Сделаем функцию для наших моделей которая покажет нам F1-метрику.

In [ ]:
def predict_model(model):
  model.fit(tf_idf_train, target_train)
  predict = model.predict(tf_idf_test)
  return predict, f1_score(target_test, predict)

Глянем на логистическую регрессию.

In [ ]:
model_logist = LogisticRegression(random_state=12345, class_weight='balanced')
predict, score = predict_model(model_logist)
score

0.7502492974345029

Получили метрику 0.75. Неплохо!

Сделаем XGBoost

In [ ]:
model_xgb = xgb.XGBClassifier(learning_rate=0.5, max_depth=4, n_estimators=250, random_state=12345)
predict, score = predict_model(model_xgb)
score

0.7617251944378978

Получили 0.76

Глянем на LGBM.

In [ ]:
model_lgbm = lgb.LGBMClassifier(boosting_type='dart', random_state=12345, learning_rate=0.5, n_estimators=250)
predict, score = predict_model(model_lgbm)
score

0.7739686228936664

Получил метрику 0.77. Это удовлетворяет ТЗ. Глянем на общую инфу и глянем на площадь под кривой.

In [ ]:
print(classification_report(target_test, predict))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     43004
           1       0.89      0.68      0.77      4868

    accuracy                           0.96     47872
   macro avg       0.93      0.84      0.88     47872
weighted avg       0.96      0.96      0.96     47872



In [ ]:
roc_auc_score(target_test, model_lgbm.predict_proba(tf_idf_test)[:,1])

0.962017752815335

## Выводы

Мы сделали несколько моделей и лучше показала себя LGBM. Получили метрику F1 - 0.77, что удовлетворяет ТЗ. Precision показала 0.89, что говорит о том, что ошибок 2 рода у нас всего 11%, а ошибок 1 рода 3%. Площадь под кривой = 0.96, что говорит о том, что наша модель хорошо научилась отделять классы друг от друга.